In [1]:
from functools import reduce
import os
from operator import or_, and_

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dateutil.parser import parse
import django
from django.db.models import Q
from flask_caching import Cache
#from jupyter_plotly_dash import JupyterDash
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from toolz import merge, isiterable, get_in

# to make jupyter-plotly-dash work, had to install jupyter_server_proxy explicitly
# pip install jupyter_server_proxy
# jupyter serverextension enable jupyter_server_proxy
# but honestly it sort of isn't very good anyway

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from plotter.models import *
# from plotter.views import *
# from plotter.forms import *
from utils import rows, columns, eta_methods, keygrab

In [2]:
app = dash.Dash()
CACHE_CONFIG = {
    'CACHE_TYPE': 'filesystem',
    'CACHE_DIR': './.cache'
}
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)

# we're using this to create thread-safe global values.
# also caching relatively expensive things like database lookups to the filesystem.
# is this actually slower? maybe. if so we will supplement with storing
# data in divs.

def cset(key, value):
    cache.set(key, value)

def cget(key):
    return cache.get(key)

In [3]:
def main_graph():
    return dcc.Graph(
            id = 'main-graph',
            figure = go.Figure(),
            style={'height':'100vh'}
        )

In [4]:
def qlist(queryset, attribute):
    return list(queryset.values_list(attribute, flat=True))

def qset_axes(queryset, x_val, y_val):
    """
    make a pair of flat lists from a queryset based on passed fields.
    for stuff like plots
    this can be made much fancier
    """
    return [qlist(queryset, x_val), qlist(queryset, y_val)]

In [5]:
def scatter(queryset, x_val, y_val):
    fig = go.Figure()
    axes = qset_axes(queryset, x_val, y_val)
    fig.add_trace(go.Scattergl(
        x = axes[0],
        y = axes[1],
        # change this to be hella popup text
        text = None,
        marker = {
            'color':'blue'
        },
        ))
    return fig

In [6]:
def flexible_query(queryset, field, value):
    """
    little search function that checks exact and loose phrases
    """
    # allow exact phrase searches
    query = field + "__iexact"
    if queryset.filter(**{query: value}):
        return queryset.filter(**{query: value})
    # otherwise treat multiple words as an 'or' search
    query = field + "__icontains"
    filters = [
        queryset.filter(**{query: word}) for word in value.split(" ")
    ]
    return reduce(or_, filters)

In [7]:
getattr(MSpec.objects.all()[0],'L2_mean')

0.0412882

In [8]:
def particular_fields_search(model, search_dict, searchable_fields):
    """
    'search specific defined fields' function.
    works only on strings atm!
    other things for numeric / date / etc. fields preferably
    
    'searchable fields' is a little clumsy -- but it specifically
    makes the function a little more fault-tolerant.
    """
    queryset = model.objects.all()
    # allow either single entries or lists of entries
    print(search_dict)
    for field in searchable_fields:
        entry = search_dict.get(field)
        if entry:
            entry = list(entry)
            filters = [flexible_query(queryset, field, value) for value in entry]
            queryset = reduce(or_,filters)
    return queryset

In [9]:
def filter_drop(model, element_id):
    """dropdown for filter selection"""
    return dcc.Dropdown(
        id=element_id,
        options=[{'label': filt, 'value': filt}
                 for filt in model.filters],
        style = {'width':'10rem', 'display':'inline-block'}
    )

In [10]:
def field_drop(fields, element_id):
    """dropdown for field selection -- no special logic atm"""
    return dcc.Dropdown(
        id=element_id,
        options = [{'label': field, 'value': field}
                 for field in fields]
    )

In [11]:
def field_values(queryset, field):
    """
    generates dict if all unique values in model's field
    + any and blank, for passing to HTML select constructors
    as this is based on current queryset,
    it will by default display options as constrained by other search
    parameters. this has upsides and downsides.
    it will also lead to odd behavior if care is not given.
    maybe it's a bad idea.
    """
    options_list = [
            {'label': item, 'value': item}
            for item in set(qlist(queryset,field))
            if item not in ['','nan']
        ]
    special_options = [
        {'label':'any',  'value':'any'},
        # too annoying to detect all 'blank' things atm
        # {'label':'no assigned value','value':''}
    ]
    return special_options + options_list


def model_options_drop(queryset, field, element_id):
    """
    dropdown for selecting search values for a specific field
    could end up getting unmanageable as a UI element
    """
    return dcc.Dropdown(
        id=element_id,
        options=field_values(queryset, field),
        multi = True
    )

In [12]:
def handle_search(model, search_dict, searchable_fields):
    """
    dispatcher. right now just handles 'no assigned'
    and 'any' cases
    """
    for field, value in search_dict.items():
        if 'any' in value:
            return model.objects.all()
    return particular_fields_search(model, search_dict, searchable_fields)

In [13]:
def handle_calc(model, calc_type, ):
    """
    dispatch function for calling calculation methods on querysets
    """
    
    
    

In [14]:
axis_value_properties = [
    {"label":"band average","value":"band_avg","type":"method","arity":2},
    {"label":"band maximum","value":"band_max","type":"method","arity":2},
    {"label":"band minimum","value":"band_min","type":"method","arity":2},
    {"label":"ratio","value":"ref_ratio","type":"method","arity":2},
    {"label":"band depth at middle filter","value":"band_depth_custom","type":"method","arity":2},
    {"label":"band depth at band minimum", "value":"band_depth_min","type":"method","arity":2},
    {"label":"band value", "value":"band_value","type":"given_property"},
    {"label":"sol", "value":"sol","type":"parent_property"}
    ]

def axis_value_drop(element_id):
    """
    dropdown for selecting calculation options for axes
    """
    options=[
            {"label":option["label"], "value":option["value"]}
             for option in axis_value_properties
        ]
    return dcc.Dropdown(
        id=element_id,
        options=options,
        value = options[0]["value"]
    )

In [15]:
for function in [
    handle_search, 
    particular_fields_search,
    flexible_query,
    qset_axes,
    qlist
    ]:
    function = cache.memoize()(function)

In [16]:
fig = main_graph()
graph_function = scatter
spec_model = MSpec 
obs_model = MObs
# update this -- a buncha fields, both spec and obs
searchable_fields = ['group','formation','member']
# active queryset is explicitly stored in global cache
cset('queryset', spec_model.objects.all())

# it's a big open UX question how to gracefully mutate filters between instruments.
# it might be that flexible band values are actually better than explicit filter
# selection.
app.layout = html.Div(children = [
    html.Div(children = [
        axis_value_drop('axis-option-x'),
        filter_drop(spec_model,'filter-1-x'),
        filter_drop(spec_model,'filter-3-x'),
        filter_drop(spec_model,'filter-2-x'),
    ]),
    html.Div(children = [
        axis_value_drop('axis-option-y'),
        filter_drop(spec_model,'filter-1-y'),
        filter_drop(spec_model,'filter-3-y'),
        filter_drop(spec_model,'filter-2-y'),
    ]),
    # will probably want to split divs off here so we can generate new search rows
    # how do we manage that in callbacks? is there a name selector?
    # yes -- see https://dash.plotly.com/pattern-matching-callbacks
    field_drop(searchable_fields, 'field-search'),
    model_options_drop(cget('queryset'), 'group', 'value-search'),
    html.Button(id='submit-search', n_clicks=0, children='Submit'),
    html.Div(children = [fig]),
    html.Div(id = 'search-signal', style={"display":"none"}),
])

In [17]:


# @app.callback(
#     dash.dependencies.Output('main-graph', 'figure'),
#     [dash.dependencies.Input('filter-1', 'value'),
#      dash.dependencies.Input('filter-2', 'value'),
#      dash.dependencies.Input('update-signal', 'value'),
#     ])
# def update_graph_calculation(filt, filt2):
#     # move axes construction to a different function
#     queryset = cget("queryset")
#     return graph_function(queryset, filt+'_mean', filt2+'_mean')

def change_input_visibility(calc_type):
    """
    turn visibility of filter dropdowns (and later other inputs)
    on and off in response to changes in arity / type of 
    requested calc
    """
    props = keygrab(axis_value_properties, "value", calc_type)
    # 'methods' are specifically those methods of spec_model
    # that take its filters as arguments
    if props["type"] == "method":
        return [
            {'width':'10rem', 'display':'inline-block'} if x < props["arity"]
            else {'width':'10rem', 'display':'none'}
            for x in range(3)
        ]
    return [
        {"display":"none"} for x in range(3)
    ]

for axis in ['x', 'y']:
    app.callback(
        [
            dash.dependencies.Output('filter-1-'+axis, 'style'),
            dash.dependencies.Output('filter-2-'+axis, 'style'),
            dash.dependencies.Output('filter-3-'+axis, 'style'),
        ],
        [dash.dependencies.Input('axis-option-'+axis, 'value')]
    )(change_input_visibility)


    
@app.callback(
    dash.dependencies.Output('main-graph', 'figure'),
    # is there a cleaner way to do this?
    # flow control becomes really hard if we break the function up
    [dash.dependencies.Input('filter-1-x', 'value'),
     dash.dependencies.Input('filter-2-x', 'value'),
     dash.dependencies.Input('filter-3-x', 'value'),
     dash.dependencies.Input('filter-1-y', 'value'),
     dash.dependencies.Input('filter-2-y', 'value'),
     dash.dependencies.Input('filter-3-y', 'value'),
     dash.dependencies.Input('search-signal', 'value'),
     dash.dependencies.Input('axis-option-x', 'value'),
     dash.dependencies.Input('axis-option-y', 'value')
     ])
def update_graph(*args):
    queryset = cget("queryset")

    # memoizing all these calculations _should_ mean that
    # we can get away with ignoring a lot of uncomfy
    # switching logic 
    print(args)
    
    #return graph_function(queryset, filt+'_mean', filt2+'_mean')


@app.callback(Output('value-search', 'options'),
              [Input('field-search', 'value')],
             )
def update_model_field(field):
    """populate set of values when field in search box changes"""
    print(field)
    queryset = cget("queryset")
    print(field_values(queryset, field))
    return field_values(queryset, field)

    
@app.callback(Output('search-signal', 'value'),
              [Input('submit-search', 'n_clicks')],
              [State('field-search', 'value'),
               State('value-search', 'value')])
def update_queryset(n_clicks, field, value):
    """
    updates the spectra displayed in the graph view.
    
    we'd actually like to extend this to include fields from both spec
    and obs
    
    and multiple fields
    """
    # don't do anything on page load
    # or if a partially blank request is issued
    if not (field and value):
        return
    if handle_search(
        spec_model, {field:value}, [field]
    ) != cget("queryset"):
        cset(
            "queryset", 
            handle_search(spec_model, {field:value}, [field])
        )
        return n_clicks
    
    

In [ ]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jun/2020 04:41:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2020 04:41:12] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2020 04:41:12] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2020 04:41:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2020 04:41:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2020 04:41:12] "POST /_dash-update-component HTTP/1.1" 200 -


None
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/michael/.local/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/michael/.local/lib/python3.8/site-packages/dash/dash

127.0.0.1 - - [28/Jun/2020 04:41:12] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [28/Jun/2020 04:41:12] "POST /_dash-update-component HTTP/1.1" 200 -


(None, None, None, None, None, None, None, 'band_avg', 'band_avg')


127.0.0.1 - - [28/Jun/2020 04:41:14] "POST /_dash-update-component HTTP/1.1" 200 -


member
[{'label': 'any', 'value': 'any'}, {'label': 'Blunts Point', 'value': 'Blunts Point'}, {'label': 'Pettegrove Point', 'value': 'Pettegrove Point'}]


127.0.0.1 - - [28/Jun/2020 04:41:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2020 04:41:38] "POST /_dash-update-component HTTP/1.1" 200 -


{'member': ['Blunts Point']}
{'member': ['Blunts Point']}
(None, None, None, None, None, None, 1, 'band_avg', 'band_avg')


In [ ]:
# ok so something that wraps querysets for plotly?
# we want both the requested attributes and at least the id,
# name, etc.
# so that people can query more information about chosen spectra

In [ ]:
queryset = cget("queryset")
field_values(queryset, "member")